In [1]:
import pandas as pd
import numpy as np

tags = pd.read_csv('tags.csv')
rating = pd.read_csv('ratings.csv')
to_read = pd.read_csv('to_read.csv')
book = pd.read_csv('book_no_null.csv')
book_tags = pd.read_csv('book_tags.csv')

## AssociationRules

In [2]:
to_read.head()

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [3]:
rating.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [4]:
to_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912705 entries, 0 to 912704
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  912705 non-null  int64
 1   book_id  912705 non-null  int64
dtypes: int64(2)
memory usage: 13.9 MB


In [5]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [6]:
to_read.isna().sum()

user_id    0
book_id    0
dtype: int64

In [7]:
rating.isna().sum()

user_id    0
book_id    0
rating     0
dtype: int64

### creating one hot from to_read & ratings datasets

In [8]:
read_1_hot = pd.DataFrame(0,columns=[i for i in range(1,10001)],index=[i for i in range(1,53425)])

In [9]:
for _uid_ in to_read['user_id'].unique():
    x = to_read[to_read['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        read_1_hot.loc[_uid_, _bid_] = 1

In [11]:
for _uid_ in rating['user_id'].unique():
    x = rating[rating['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        read_1_hot.loc[_uid_, _bid_] = 1

In [27]:
read_1_hot

,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,1,1,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
53421,1,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
53422,1,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
53423,1,1,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
read_1_hot.to_csv('to_read_1_hot.csv')

### which books are related

In [108]:
import pandas as pd
import numpy as np

read_1_hot = pd.read_csv('to_read_1_hot.csv')
read_1_hot.drop(['Unnamed: 0'], axis=1, inplace=True)

In [109]:
read_1_hot.head()

,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [147]:
frq_items_fpgrowth = fpgrowth(read_1_hot, min_support = 0.1, use_colnames = True)

frq_items_fpgrowth.sort_values(by=['support'], inplace=True)

In [148]:
frq_items_fpgrowth.shape

(2071, 2)

In [149]:
frq_items_fpgrowth

,support,itemsets
1735,0.100011,"(27, 2, 18, 26)"
1004,0.100011,"(23, 18, 20, 21, 25, 2)"
1227,0.100011,"(23, 21, 7, 27, 2)"
1586,0.100030,"(23, 20, 21, 27, 25, 2)"
696,0.100030,"(20, 17, 25, 24, 2, 1)"
...,...,...
120,0.322271,(3)
58,0.334924,(5)
0,0.384864,(4)
57,0.416348,(2)


In [150]:
book_rules = association_rules(frq_items_fpgrowth, metric ="lift", min_threshold = 1)

In [151]:
book_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(27, 2, 18)",(26),0.215690,0.253051,0.100011,0.463681,1.832363,0.045431,1.392733
1,"(27, 2, 26)",(18),0.104541,0.304676,0.100011,0.956670,3.139959,0.068160,16.047048
2,"(27, 18, 26)",(2),0.116783,0.416348,0.100011,0.856387,2.056900,0.051389,4.064065
3,"(2, 26, 18)",(27),0.110344,0.290450,0.100011,0.906361,3.120542,0.067962,7.577531
4,"(27, 2)","(18, 26)",0.227782,0.130746,0.100011,0.439066,3.358151,0.070230,1.549655
...,...,...,...,...,...,...,...,...,...
44447,(18),(23),0.304676,0.300558,0.276655,0.908030,3.021148,0.185082,7.605091
44448,(18),(24),0.304676,0.298518,0.276767,0.908398,3.043032,0.185816,7.657968
44449,(24),(18),0.298518,0.304676,0.276767,0.927138,3.043032,0.185816,9.543055
44450,(17),(20),0.320792,0.311339,0.287062,0.894854,2.874205,0.187187,6.549536


In [116]:
book_rules.to_csv('book_rules.csv')

### creating to_read_authors dataset using to_read & ratings datasets

In [21]:
import re
unique_authors = set()
for i in book['authors'].unique():
    res = re.split(',', i)
    for j in res:
        unique_authors.add(j.lstrip())

In [22]:
len(unique_authors)

5841

In [23]:
authors_1_hot = pd.DataFrame(0,columns=[i for i in unique_authors],index=[i for i in range(0,53425)])

In [24]:
for _uid_ in to_read['user_id'].unique():
    x = to_read[to_read['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        authors = re.split(',', book.loc[_bid_ -1, 'authors'])
        for _au_ in authors:
            authors_1_hot.loc[_uid_, _au_.lstrip()] = 1

In [25]:
for _uid_ in rating['user_id'].unique():
    x = rating[rating['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        authors = re.split(',', book.loc[_bid_ -1, 'authors'])
        for _au_ in authors:
            authors_1_hot.loc[_uid_, _au_.lstrip()] = 1

In [26]:
authors_1_hot

,Banesh Hoffmann,Andrew Loomis,Claire LaZebnik,Alexandra Boiger,Emma Goldman,Michael Baigent,Seth Grahame-Smith,Diane Dillon,Steven Hall,David Brooks,...,Shannon Hale,Patrick Rothfuss,Jim Fergus,Louis de Bernières,Sarah Winman,Gena Showalter,Kristen Simmons,Joshua Ferris,Dan Chaon,Thor Heyerdahl
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53421,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
53422,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
authors_1_hot.to_csv('authors_1_hot.csv')

### which authors are related

In [1]:
import pandas as pd
import numpy as np

authors_1_hot = pd.read_csv('authors_1_hot.csv')
authors_1_hot.drop(['Unnamed: 0'], axis=1, inplace=True)

In [132]:
authors_1_hot.head()

,Banesh Hoffmann,Andrew Loomis,Claire LaZebnik,Alexandra Boiger,Emma Goldman,Michael Baigent,Seth Grahame-Smith,Diane Dillon,Steven Hall,David Brooks,...,Shannon Hale,Patrick Rothfuss,Jim Fergus,Louis de Bernières,Sarah Winman,Gena Showalter,Kristen Simmons,Joshua Ferris,Dan Chaon,Thor Heyerdahl
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [134]:
frq_items_fpgrowth = fpgrowth(authors_1_hot, min_support = 0.25, use_colnames = True)

frq_items_fpgrowth.sort_values(by=['support'], inplace=True)

In [135]:
frq_items_fpgrowth.shape

(72, 2)

In [136]:
frq_items_fpgrowth

,support,itemsets
28,0.250108,(Garth Williams)
63,0.252410,"(Guy Cardwell, John Seelye, Mark Twain)"
62,0.252410,"(Guy Cardwell, Mark Twain)"
61,0.252410,"(Guy Cardwell, John Seelye)"
27,0.252410,(Guy Cardwell)
...,...,...
29,0.435470,(J.R.R. Tolkien)
51,0.518390,"(J.K. Rowling, Mary GrandPré)"
17,0.518390,(Mary GrandPré)
32,0.520056,(Suzanne Collins)


In [137]:
author_rules = association_rules(frq_items_fpgrowth, metric ="lift", min_threshold = 1)

In [138]:
author_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Guy Cardwell, John Seelye)",(Mark Twain),0.252410,0.276743,0.252410,1.000000,3.613460,0.182557,inf
1,"(Guy Cardwell, Mark Twain)",(John Seelye),0.252410,0.259785,0.252410,1.000000,3.849341,0.186838,inf
2,"(John Seelye, Mark Twain)",(Guy Cardwell),0.253046,0.252410,0.252410,0.997485,3.951846,0.188539,297.255008
3,(Guy Cardwell),"(John Seelye, Mark Twain)",0.252410,0.253046,0.252410,1.000000,3.951846,0.188539,inf
4,(John Seelye),"(Guy Cardwell, Mark Twain)",0.259785,0.252410,0.252410,0.971612,3.849341,0.186838,26.334525
...,...,...,...,...,...,...,...,...,...
101,(Mary GrandPré),"(J.K. Rowling, Suzanne Collins)",0.518390,0.376172,0.335929,0.648023,1.722677,0.140925,1.772354
102,(J.K. Rowling),(Suzanne Collins),0.613552,0.520056,0.376172,0.613106,1.178923,0.057091,1.240505
103,(Suzanne Collins),(J.K. Rowling),0.520056,0.613552,0.376172,0.723330,1.178923,0.057091,1.396784
104,(J.K. Rowling),(Mary GrandPré),0.613552,0.518390,0.518390,0.844901,1.629854,0.200331,3.105170


In [139]:
author_rules.to_csv('author_rules.csv')

### Recommender

In [125]:
#author_rules = pd.read_csv('author_rules.csv')
#author_rules.drop(['Unnamed: 0'], axis=1, inplace=True)

In [126]:
#book_rules = pd.read_csv('book_rules.csv')
#book_rules.drop(['Unnamed: 0'], axis=1, inplace=True)

#### Execute 'which authors are related' and 'which books are related' befor execution of recommender

In [249]:
from itertools import chain, combinations
def powerset(iterable):
    return chain.from_iterable(combinations(iterable, r) for r in range(len(iterable)+1))

In [303]:
def recommender(read_list, df):
    
    u = frozenset(read_list)
    consequents = set()
    for item in list(powerset(read_list)):
        antecedents = set()
        for i in item:
            antecedents.add(i)
            
        for i in range(len(df)):
            each_antecedents = [x for x in df['antecedents'].loc[i]]
            each_antecedents_set = set()
            for j in each_antecedents:
                each_antecedents_set.add(j)
            
            if antecedents == each_antecedents_set:
                diff = df['consequents'].loc[i].difference(u)
                if diff != frozenset():
                    consequents.add(diff)
                                    
    all_consequents = [list(x) for x in consequents]
    recomended_list = set()
    for i in all_consequents:
        for j in i:
            recomended_list.add(j)     
    return recomended_list

In [309]:
authors_read = ["Suzanne Collins", "Mary GrandPré"]
recomended_list = recommender(authors_read, author_rules)
print(recomended_list)
print('______________________________________')
print(len(recomended_list), ' recomended item')

{'J.K. Rowling', 'J.R.R. Tolkien', 'Stephenie Meyer', 'Dan Brown', 'George Orwell', 'Rufus Beck', 'Harper Lee'}
______________________________________
7  recomended item


In [310]:
books_read = ['27', '21']
recomended_list = recommender(books_read, book_rules)
print(recomended_list)
print('______________________________________')
print(len(recomended_list), ' recomended item')

{'3', '23', '28', '8', '19', '56', '13', '20', '4', '25', '49', '52', '37', '7', '1', '14', '18', '10', '5', '26', '17', '24', '2'}
______________________________________
23  recomended item
